<div align="center">
    <h1>DS-210: Programming for Data Science</h1>
    <h1>Lecture 8</h1>
</div>

# Topics
1. Ownership and borrowing in Rust
2. Methods in Rust
3. Copying instead of moving
4. Multiple references in parallel
5. Generics
6. Useful predefined generic data types

* Chapters 4 (Understanding Ownership), 10 (Generic Types) skip 10.3 (Lifetimes)

---

## Quick Aside on Data Type Sizes

Consider the data type sizes need for the results of multiplication and adddition.

(Helpful for the homework.)

### Multiplication

`u8` is 8 bits and can store maximum value `2^8 - 1 = 255`.

If we multiply: $255*255=65025$. 

How many bits do we need to store this value? We can take the log base 2 of the value.

```python
>>> import math
>>> math.log2(255*255)
15.988706873717716
```

So we need 16 bits to store the product of two `u8` values.

In general when we multiply two numbers of size $n$ bits, we need $2n$ bits to store the result.

### Addition

How about addition?

Consider a `u8` with the value 255 stored in it.

What if we add 255 to it? $255 + 255 = 510$.

```python
>>> math.log2(510)
8.994353436858859
```

We'll need 9 bits to store the result of the addition.

---

## Ownership and borrowing in Rust

* We've used `String` and `Vec` so far to store data on the heap
* These are two examples of data types that are **smart pointers** (§15)
* A **pointer** is a general concept for a _variable_ that contains a memory address. **Not in Rust!!**

<div style="text-align: center;">
  <img src="pointer.png" alt="Simple pointer" style="width: 40%;">
</div>
<div style="text-align: center;">
  <i>Example of a simple pointer. Note: does not exist in Rust.</i>
</div>

**Smart pointers** are pointers that have additional metadata that allows the
compiler to do more checks and optimizations.

We saw an example of that in the previous lecture with a `String` that contains a
structure on the stack with a pointer to a heap-allocated string.

<div style="text-align: center;">
  <img src="string-data-structures.png" alt="String data structures" style="width: 40%;">
</div>
<div style="text-align: center;">
  <i>Example of a `String` data structure and associated memory on the heap.</i>
</div>

### `Box<T>`

Rust provides a datatype called `Box<T>` that is a general purpose smart pointer
that lets us allocate a single value on the heap.

```rust
let mut pointer = Box::new(2000);
```

This creates a new `Box<i32>` on the heap with the value `2000`.



Why is it helpful to allocate a single value on the heap?

There can be cases when we don't know the size of the datatype at compile time.


Let's look at examples of using `Box<T>` to store data on the heap.

In [2]:
let val: u8 = 5;  // store 5 on the stack
let boxed: Box<u8> = Box::new(val);  // store 5 on the heap

println!("boxed: {:?}", boxed);
println!("val: {:?}", val);


boxed: 5


In [4]:
let boxed: Box<u8> = Box::new(5);  // store 5 on the heap

// can we copy the value back to the stack?
let val: u8 = boxed;


println!("boxed: {:?}", boxed);
println!("val: {:?}", val);


Error: mismatched types

The variable `boxed` is a data structure that contains a pointer to the heap-allocated value.

We have to _dereference_ the pointer to get the value using `*boxed`.

---

### Optional: Linked lists using `Box<T>`

Here's an example of a kind of linked list that uses `Box<T>` to store the data on the heap.

See [Module boxed Examples](https://doc.rust-lang.org/std/boxed/index.html).

In [5]:
#[allow(dead_code)]
#[derive(Debug)]
enum List<T> {
    Cons(T, Box<List<T>>),
    Nil,
}

let list: List<i32> = List::Cons(1, Box::new(List::Cons(2, Box::new(List::Nil))));
println!("{list:?}");

Cons(1, Cons(2, Nil))


We can't just recursively define a list like this:

In [6]:
// ERROR

#[allow(dead_code)]
#[derive(Debug)]
enum List<T> {
    Cons(T, List<T>),
    Nil,
}

let list: List<i32> = List::Cons(1, List::Cons(2, List::Nil));
println!("{list:?}");

Error: recursive type `List` has infinite size

Error: cycle detected when computing when `List` needs drop

---

Let's allocate a `Box<i32>` on the heap and print the pointer to it and it's containing structure.

In [7]:
let boxed: Box<i32> = Box::new(2000);
println!("&boxed: {:p}", &boxed);
println!("into_raw(boxed): {:p}", Box::<i32>::into_raw(boxed));


&boxed: 0x16afc9620


In [9]:
// store 2000 and 22 on the heap
let mut pointer = Box::new(2000);
let pointer2 = Box::new(22);

println!("pointer: {:?}, &pointer: {:p}, pointer.as_ref(): {:p}", pointer, &pointer, pointer.as_ref());
println!("pointer2: {:?}, &pointer2: {:p}, pointer2.as_ref(): {:p}", pointer2, &pointer2, pointer2.as_ref());

println!("\nClone pointer2 into pointer");

// We can't directly assign a `Box<i32>` to another `Box<i32>`
//pointer = pointer2;  // try this to see what happens

// We can clone a `Box<i32>` and assign it to another `Box<i32>`
pointer = pointer2.clone();

println!("pointer: {:?}, &pointer: {:p}, pointer.as_ref(): {:p}", pointer, &pointer, pointer.as_ref());
println!("pointer2: {:?}, &pointer2: {:p}, pointer2.as_ref(): {:p}", pointer2, &pointer2, pointer2.as_ref());


pointer: 2000, &pointer: 0x16afc95b0, pointer.as_ref(): 0x14d7040a0
pointer2: 22, &pointer2: 0x16afc95b8, pointer2.as_ref(): 0x14d704110

Clone pointer2 into pointer
pointer: 22, &pointer: 0x16afc95b0, pointer.as_ref(): 0x14d704080
pointer2: 22, &pointer2: 0x16afc95b8, pointer2.as_ref(): 0x14d704110


Pay attention to what happened to the pointers after the clone.

The structure containing the pointer is the same, but the pointer to the heap is different.

In [10]:
let mut pointer = Box::new(2000);
println!("pointer: {:?}", pointer);

let z = pointer.clone();
println!("z: {:?}", z);

// when a variable goes out of scope, it is dropped
// we can use the drop function to explicitly drop a variable
// this is why we can't use pointer after this point
drop(pointer);
//println!("pointer:{:?}", pointer);  // try to uncomment this to see what happens
println!("z: {:?}", z);

pointer: 2000
z: 2000
z: 2000


## Experiment with passing the pointer around

In [11]:
fn print_content(pointer: Box<i32>) {
    println!("print_content: {}", *pointer); // dereference
    println!("print_content: {}", pointer);  // actually rust is smart enough to dereference
    println!("print_content: {:p}", &pointer);  // prints a pointer to the containing structure
    println!("print_content: {:p}", pointer);  // but naturally prints a pointer to heap
    println!("print_content: {:p}", pointer.as_ref());

}

fn print_value(val: i32){
    println!("print_value: {}", val);
    // println!("print_value: {:p}", val);  // this won't work
    println!("print_value: {:p}", &val);
}

let p = Box::new(123);
// print_value(p); // ERROR: why doesn't this work?
print_value(*p);
println!("");

print_content(p); 

print_value: 123
print_value: 0x16afc9604

print_content: 123
print_content: 123
print_content: 0x16afc9570
print_content: 0x14e004100
print_content: 0x14e004100


In [13]:
let q = Box::new(321);

print_content(q);
//print_content(q); // uncomment this to see what happens

print_content: 321
print_content: 321
print_content: 0x16afc95f0
print_content: 0x14e0040a0
print_content: 0x14e0040a0


## What happened: Ownership

* Each value in Rust has a variable that is its **owner**
* Only **one** owner
* When the owner goes out of scope, the value is dropped

In [14]:
fn print_content(pointer:Box<i32>) {   // we move ownership of pointer to the function
    println!("content: {}", *pointer)
}  // pointer goes out of scope here and the value is dropped

let q = Box::new(321);

print_content(q);
print_content(q);  // ERROR: why doesn't this work?

Error: use of moved value: `q`

* First call to `print_content`: `Box::new(321)` is **moved** from `q` to `pointer`
* (if it compiled) at the end of `print_content`:
  * `Box::new(321)` would be dropped
  * its space on the heap deallocated

Second call can't proceed: **the content of `q` is gone**

## More examples of ownership

In [15]:
// won't work, value moved as well
let x = Box::new(123);
println!("x = {}",*x);

let y = x;
println!("x = {}",*x);

Error: borrow of moved value: `x`

Fix our previous example by returning the pointer

In [16]:
fn print_content(pointer:Box<i32>) -> Box<i32> {
    println!("content: {}", *pointer);
    return pointer;
}

let q = Box::new(321);

let q = print_content(q);
let q = print_content(q);
let q = print_content(q);

content: 321
content: 321
content: 321


## Avoiding moving values a lot: borrowing -- Part 1

**Passing and returning values**


In [17]:
#[derive(Debug)]
struct Road {
    intersection_1: u32,
    intersection_2: u32,
    max_speed: u32,
}

// adding a function in the namespace of Road
impl Road {
    // very useful constructor
    fn new(i1:u32,i2:u32,speed:u32) -> Road {
        Road {
            intersection_1: i1,
            intersection_2: i2,
            max_speed: speed,
        }
    }
}

let road = Road::new(13,23,25);
println!("{:?}",road);

Road { intersection_1: 13, intersection_2: 23, max_speed: 25 }


Remember that we can create a new instance of a struct by using the `struct_name` and then the field names,
rather than using our constructor method.

In [18]:
let road2 = Road{intersection_1: 101, intersection_2: 102, max_speed: 30};
println!("{:?}",road2);

Road { intersection_1: 101, intersection_2: 102, max_speed: 30 }


Let's try assignning `road` to another variable and then try accessing it.

In [19]:
// ERROR
// checking whether it moves
let another = road;
println!("{}",road.max_speed);

Error: borrow of moved value: `road`

The error message is informative. We actually moved ownership of `road` to `another`.

In [20]:
fn display_1(r:Road) {     // ownership of r is moved to the function
    println!("{:?}",r);
}  // r goes out of scope here and the value is dropped

let road = Road::new(101,102,30);
display_1(road);
// Uncomment this one to see ownership behavior
// display_1(road);  

Road { intersection_1: 101, intersection_2: 102, max_speed: 30 }


In [21]:
// This one works due to returning the value
fn display_1(r:Road) -> Road {
    println!("{:?}",r);
    r
}

let mut road = Road::new(101,102,30);
road = display_1(road);
road = display_1(road);

Road { intersection_1: 101, intersection_2: 102, max_speed: 30 }
Road { intersection_1: 101, intersection_2: 102, max_speed: 30 }


But we're basically moving the variable to the function and then returning the value as a new variable.

There's a better way to do this: borrowing.

## Avoiding moving values a lot: borrowing -- Part 2

**Read-only reference**

* Reference type becomes `&Type`
* To create: `&value`
* To access content: `*reference`

In [22]:
fn display_2(r:&Road) {
    println!("{:?}", *r); // This is the right one
    println!("{:?}", r);  // In this case the compiler does the conversion for you but it may not work all times.
    println!("{:?}", &r); // Ditto here.
}

let road = Road::new(101,102,30);
display_2(&road); // <- have to explicitly create a reference
display_2(&road);

Road { intersection_1: 101, intersection_2: 102, max_speed: 30 }
Road { intersection_1: 101, intersection_2: 102, max_speed: 30 }
Road { intersection_1: 101, intersection_2: 102, max_speed: 30 }
Road { intersection_1: 101, intersection_2: 102, max_speed: 30 }
Road { intersection_1: 101, intersection_2: 102, max_speed: 30 }
Road { intersection_1: 101, intersection_2: 102, max_speed: 30 }


## Avoiding moving values a lot: borrowing -- Part 3

**Mutable reference**

* Reference type becomes `&mut Type`
* To create: `&mut value`
* To access content: `*reference`

Previously we were passing _immutable_ references to the function.

That was ok because we were only reading the data.

Now let's try to update the data.

In [23]:
// regular (immutable) references won't work
fn update_speed(r:&Road, new_speed: u32) {
    // r.max_speed equivalent to (*r).max_speed
    // because Rust is smart
    r.max_speed = new_speed;
} 

Error: cannot assign to `r.max_speed`, which is behind a `&` reference

In [24]:
// Change the function to take a mutable reference
fn update_speed(r:&mut Road, new_speed: u32) {
    // r.max_speed equivalent to (*r).max_speed
    // because Rust is smart
    r.max_speed = new_speed;
} 

In [25]:
fn display_3(r:&Road) {
    println!("{:?}", *r);
}

let mut road = Road::new(100,200,30);
display_3(&road);
update_speed(&mut road, 25);
display_3(&road);

Road { intersection_1: 100, intersection_2: 200, max_speed: 30 }
Road { intersection_1: 100, intersection_2: 200, max_speed: 25 }


## Revisit Methods

We saw these in the previous lecture.


* We can add functions that are directly associated with structs and enums!
  * Then we could call them: `road.display()` or `road.update_speed(25)`
* How?
  * Put them in the namespace of the type
  * make `self` the first argument

In [26]:
#[derive(Debug)]
struct Road {
    intersection_1: u32,
    intersection_2: u32,
    max_speed: u32,
}

impl Road {
    
    // constructor
    fn new(i1:u32,i2:u32,speed:u32) -> Road {
        Road {
            intersection_1: i1,
            intersection_2: i2,
            max_speed: speed,
        }
    }
    // note &self: immutable reference
    fn display(&self) {
        println!("{:?}",*self);
    }
}

You can invoke the display method on the road instance or on a reference to the road instance.



In [27]:
let mut road = Road::new(1,2,35);

road.display();
&road.display();
(&road).display();

Road { intersection_1: 1, intersection_2: 2, max_speed: 35 }
Road { intersection_1: 1, intersection_2: 2, max_speed: 35 }
Road { intersection_1: 1, intersection_2: 2, max_speed: 35 }


> In C++ the syntax is different. It would be something like:
> 
> ```cpp
> road.display();
> ```
> 
> ```cpp
> (&road)->display();
> ```

## Methods (continued)

Rember that `self` is a reference to the instance of the struct.

By default, `self` is an immutable reference, so we can't modify the struct.


In [28]:
// ERROR
impl Road {
    fn n_update_speed(&self, new_speed:u32) {
        self.max_speed = new_speed;
    }
}

Error: cannot assign to `self.max_speed`, which is behind a `&` reference

Let's change it to a mutable reference.

In [29]:
impl Road {
    fn update_speed(&mut self, new_speed:u32) {
        self.max_speed = new_speed;
    }
}

In [30]:
road.display();
road.update_speed(45);
road.display();

Road { intersection_1: 1, intersection_2: 2, max_speed: 35 }
Road { intersection_1: 1, intersection_2: 2, max_speed: 45 }


There are some gotchas to be aware of.

Consider the following code:

In [31]:
impl Road {
    
    fn this_will_move(self) -> Road {   // this will take ownership of the instance of Road
        self
    }
    
    fn this_will_not_move(&self) -> &Road {
        self
    }
}

In [32]:
#![allow(unused_variables)]
fn testme() {
  let r = Road::new(1,2,35);       // create a new instance of Road, r
  let r3 = r.this_will_not_move(); // create a new reference to r, r3

  // run the code with the following line commented, then try uncommenting it
  // let r2 = r.this_will_move();  // this will take ownership of r

  r.display();

  // r2.display();
  r3.display();
}
testme();

Road { intersection_1: 1, intersection_2: 2, max_speed: 35 }


## Methods (summary)

* Make first parameter `self`
* Various options:
  - `self`: move will occur
  - `&self`: self will be immutable reference
  - `&mut self`: self will be mutable reference

## A few more pointer types

In [34]:
// A few more pointer types for future reference

use std::rc::Rc;
use std::cell::RefCell;

// A reference counted pointer.  Behaves like a box pointer in this example
let mut pointer = Rc::new(12);
let pointer2 : Rc<i32> = Rc::new(22);
println!("sum: {}", *pointer + *pointer2);

// But as we saw you can not assigned box pointers to each other without clone
let mut pointer = Box::new(10);
let mut pointer2 = pointer.clone();
*pointer2 = 22;
println!("Box Vals {} {} ", *pointer, *pointer2);


// But Rc pointers with refcells allow for this, 
// though you have to tell the compiler a lot of information to allow it
let mut pointer = Rc::new(RefCell::new(10));
let mut pointer2 = Rc::new(Rc::clone(&pointer));
*pointer.borrow_mut() = 22;
println!("Ref Vals {} {} ", *pointer.borrow(), *pointer2.borrow());

sum: 34


Box Vals 10 22 
Ref Vals 22 22 


## Cloning and Copying


Let's define a `BoxSize` struct and a constructor method.

In [35]:
#[derive(Debug)]
struct BoxSize {
    height: f64,
    width: f64,
    depth: f64,
}

impl BoxSize {
    fn new(height: f64, width: f64, depth: f64)
        -> BoxSize {
        BoxSize{
            height: height, 
            width, // = width: width (shorthand syntax)
            depth, // = depth: depth (shorthand syntax)
        }
    }
}

Simple built-in types like `i32` and `f64` are copied by default.

But can we copy a `BoxSize` struct?


In [36]:
let x = 5;
println!("{}",x);
let y = x;
println!("{}",x);

let xl_box = BoxSize::new(24.0,18.0,24.0);
println!("{:?}", xl_box);
let move_it_here = xl_box;   // this will move the instance of BoxSize to move_it_here
println!("{:?}", xl_box);    // error: xl_box is no longer valid

Error: borrow of moved value: `xl_box`

<div align="center">
<img src="uhaul_xl.png" alt="[image of a box]">
</div>

## Cloning

How to make a copy of data?

**Option 1:** Implement yourself

In [37]:
impl BoxSize {
    fn give_me_a_copy(&self) -> BoxSize {
        let BoxSize{height,width,depth} = *self;  // manual implementation of clone logic
        BoxSize{height,width,depth}
    }
}

In [38]:
let box_1 = BoxSize::new(1.1,2.2,3.3);
println!("box_1: {:?}",box_1);
let box_2 = box_1.give_me_a_copy();
println!("================");
println!("box_1: {:?}",box_1);
println!("box_2: {:?}\n",box_2);
// Also
println!("&box_1: {:?}",&box_1);
println!("&box_2: {:?}\n",&box_2);
// But 
println!("&box_1: {:p} &box_2: {:p}", &box_1, &box_2)

box_1: BoxSize { height: 1.1, width: 2.2, depth: 3.3 }
box_1: BoxSize { height: 1.1, width: 2.2, depth: 3.3 }
box_2: BoxSize { height: 1.1, width: 2.2, depth: 3.3 }

&box_1: BoxSize { height: 1.1, width: 2.2, depth: 3.3 }
&box_2: BoxSize { height: 1.1, width: 2.2, depth: 3.3 }

&box_1: 0x16afc9558 &box_2: 0x16afc9570


()

## Cloning

**Option 2:** Default cloning (with some extra benefits)
* Use `#[derive(Clone)]` in the definition
* Use method `.clone()` to clone an object

In [39]:
#[derive(Clone,Debug)]
struct CloneablePoint {
    x: f64,
    y: f64,
}

In [40]:
let point_1 = CloneablePoint{x:2.2,y:-1.4};
let point_2 = point_1.clone();
println!("{:?}\n{:?}",point_1,point_2);

CloneablePoint { x: 2.2, y: -1.4 }
CloneablePoint { x: 2.2, y: -1.4 }


Can then be used recursively:

In [41]:
// will work
let tuple_point = (1,CloneablePoint{x:1.1,y:1.1});
let copy_tuple_point = tuple_point.clone();

In [42]:
// won't work
let tuple_box = (1,BoxSize::new(1.1,1.2,1.3));
let copy_tuple_box = tuple_box.clone();

Error: the method `clone` exists for tuple `({integer}, BoxSize)`, but its trait bounds were not satisfied

## Implicit copying

* Works for intergers, floats, booleans, ...
* Also for tuples made of items for which it works
* But not for other types including those that implement clone

In [43]:
let int = 3;
let int_2 = int;
println!("{}\n{}",int,int_2);

3
3


In [44]:
let tuple = (1.2, 3.1);
let tuple_2 = tuple;
println!("{:?}\n{:?}",tuple,tuple_2);

(1.2, 3.1)
(1.2, 3.1)


To make it work: use `#[derive(Copy)]` in the definition
* `(Clone)` needed as well

In [45]:
#[derive(Copy,Clone,Debug)]
enum SearchResult {
    DidntFindIt,
    FoundIt(usize),
}

In [46]:
let result = SearchResult::DidntFindIt;
let will_it_move = result;

println!("{:?}\n{:?}",result,will_it_move);

DidntFindIt
DidntFindIt


## What really happens with `derive(Copy)` and `derive(Clone)`

* Defining a specific method or methods (i.e., `clone`)

* It tells Rust that the type meets specific requirements 
  * they are called a trait
  * to be covered later in class (next lecture)

## Multiple immutable references at once

* useful for when we may want to access the same thing from multiple places
* they can be passed around like values

In [47]:
// auxiliary functions

fn display(x:&i32) {
    println!("{}",*x);    
}

fn double(x:&mut i32) {
    *x *= 2;
}

In [48]:
// two immutable references
let mut integer = 1;
{
    integer = 5;
    let ir = &integer;
    let ir2 = &integer;
    display(ir);
    display(ir2);
    println!("integer: {:p}", &integer);
    println!("ir: {:p} ir2: {:p}", ir, ir2);
};

5
5
integer: 0x16afc94a4
ir: 0x16afc94a4 ir2: 0x16afc94a4


These are references to the same instance of `integer`.

## Multiple mutable references at once

* useful for when we may want to access the same thing from multiple places
* they can be passed around like values

In [49]:
// one mutable reference
{
    let mr = &mut integer;
    double(mr);
    display(mr);
};

10


But we can't have two mutable references at once.

In [50]:
// ERROR
// two mutable references
{
    let mr = &mut integer;
    let mr2 = &mut integer;
    double(mr);
    double(mr2);
};

Error: cannot borrow `integer` as mutable more than once at a time

## Multiple references at once -- mutable and immutable

* useful for when we may want to access the same thing from multiple places
* they can be passed around like values

We can't take a mutable reference after an immutable reference.

In [51]:
// immutable and mutable references
{
    let ir = &integer;
    let mr2 = &mut integer;
    display(ir);  // borrowing immutable reference
    double(mr2);
};

Error: cannot borrow `integer` as mutable because it is also borrowed as immutable

What about below?

In [52]:
// immutable and mutable references
//#![allow(unused_variables)]
{
    let ir = &integer;
    display(ir);
    let mr2 = &mut integer;
    double(mr2);
    let ir2 = &integer;
    display(ir2);
};

10


Rust can figure out which references no longer used

## Multiple references at once

* useful for when we may want to access the same data from multiple places
* they can be passed around like values

### Rules
* At most one mutable reference at a time
* Multiple immutable references allowed
* No mutable and immutable references at the same time (execution order matters)

### How it could be useful
* More clear what is happening
  * Potential early bug detection
* Additional optimizations possible
* Multithreading (running things in parallel):
  * each thread accesses things through references
  * potentially very unpredictable behaviour
    without these rules

## Not covered today: lifetimes
* how long a reference lives
* important for making sure that references passed around are not in conflict
* useful for dealing with some data processing patterns

---

# Generics

## New Topic: Avoiding duplicating code for different types

Python:

```python
def max(x,y):
    return x if x > y else y
```
```python
max(3,2)
```
```
3
```
```python
max(3.1,2.2)
```
```
3.1
```

Very flexible! Any downsides?

* Requires checking each time what types are used
* Runtime penalty

## New Topic: Avoiding duplicating code for different types

Possible Rust "equivalent": create a version of the function for each type

In [54]:
fn max_i32(x:i32,y:i32) -> i32 {
    if x > y {x} else {y}
}

max_i32(3,8)

8

In [55]:
fn max_f64(x:f64,y:f64) -> f64 {
    if x > y {x} else {y}
}

max_f64(3.3,8.1)

8.1

Lots of work! Make the compiler do it!

In [56]:
fn max<T>(x:T,y:T) -> T {
        if x > y {x} else {y}
}

Error: binary operation `>` cannot be applied to type `T`

In [57]:
// add info that elements of T are comparable
fn max<T:PartialOrd>(x:T,y:T) -> T {
        if x > y {x} else {y}
}

println!("{}",max(3,8));
println!("{}",max(3.3,8.1));
println!("{}",max('a','b'));

8


## Generics / Generic data types
In other programming languages:<br>
&nbsp;&nbsp;&nbsp;&nbsp;$\bullet$ C++: templates<br>
&nbsp;&nbsp;&nbsp;&nbsp;$\bullet$ Java: generics<br>
&nbsp;&nbsp;&nbsp;&nbsp;$\bullet$ Go: generics<br>
&nbsp;&nbsp;&nbsp;&nbsp;$\bullet$ ML, Haskell: parametric polymorphism

## Use with data types

In [59]:
#[derive(Debug)]
struct Point<T> {
    x: T,
    y: T,
}

In [60]:
let point_int = Point {x: 2, y: 3};
println!("{:?}", point_int);

let point_float = Point {x: 4.2, y: 3.0};
println!("{:?}", point_float);


Point { x: 2, y: 3 }
Point { x: 4.2, y: 3.0 }


Functions and methods for generic data types

In [61]:
impl<T> Point<T> {
    fn create(x:T,y:T) -> Point<T> {
        Point{x,y}
    }
}

`impl<T> Point<T>` means that this is an implementation block and all the methods
are implemented for any type `T` that `Point` might be instantiated with.

In [62]:
let point = Point::create('a','b');
let point2 = Point::<char>::create('c','d');
let point3 : Point<char> = Point::create('c','d');
println!("{:?} {:?} {:?}", point, point2, point3);


Point { x: 'a', y: 'b' } Point { x: 'c', y: 'd' } Point { x: 'c', y: 'd' }


## Use with data types

Implementing a method

In [63]:
impl<T:Copy> Point<T> {
    fn swap(&mut self) {
        let z = self.x;
        self.x = self.y;
        self.y = z;
    }
}

`impl<T:Copy>` specifies that `T` must implement the `Copy` trait.

In [64]:
impl<T> Point<T> {
    fn swap2(&mut self) {
        let z = self.x;
        self.x = self.y;
        self.y = z;
    }
}

Error: cannot move out of `self.x` which is behind a mutable reference

Error: cannot move out of `self.y` which is behind a mutable reference

In [65]:
let mut point = Point::create(2,3);
println!("{:?}",point);
point.swap();
println!("{:?}",point);


Point { x: 2, y: 3 }
Point { x: 3, y: 2 }


## Use with data types

Specialized versions for different types

In [66]:
impl Point<i32> {
    fn do_you_use_f64(&self) -> bool {
        false
    }
}

In [67]:
impl Point<f64> {
    fn do_you_use_f64(&self) -> bool {
        true
    }
}


In [68]:
let p_i32 = Point::create(2,3);
p_i32.do_you_use_f64()

false

In [69]:
let p_f64 = Point::create(2.1,3.1);
p_f64.do_you_use_f64()

true

# <font color="red">Useful predefined generic data types</font>


* Generic code
* Method for avoiding copying code
* No runtime penalty: different versions created during compilation

Generic data types:
* Data types (struct/enum) parameterized by types

Two useful predifined types: `Option<T>` and `Result<T, E>`

## Enum `Option<T>`

There is a built-in enum 
[`Option<T>`](https://doc.rust-lang.org/book/ch06-01-defining-an-enum.html?highlight=Option%3CT%3E#the-option-enum-and-its-advantages-over-null-values)
in the standard library with two variants:

* `Some(T)`
* `None`

* Useful for when there may be no output
* Compared to `None` or `null` in other programming languages:
  * Rust forces handling of this case
  
<div align="center">
<img src="null.png" alt="[abc]" >
</div>

Here's example prime number finding code that returns `Option<u32>`:

In [70]:
fn prime(x:u32) -> bool {
    if x <= 1 { return false;}
    for i in 2..=((x as f64).sqrt() as u32) {
        if x % i == 0 {
            return false;
        }
    } 
    true
}

fn prime_in_range(a:u32,b:u32) -> Option<u32> {  // returns an Option<u32>
    for i in a..=b {
        if prime(i) {return Some(i);}
    }
    None
}

If a prime number is found, it returns `Some(u32)` variant with the prime number.

In [71]:
prime_in_range(90,906)

Some(97)

If the prime number is not found, it returns `None`.

In [72]:
prime_in_range(90,92)

None

In [73]:
let tmp : Option<u32> = prime_in_range(830,856);
tmp

Some(839)

There are various ways to extract the content of `Some(...)`
* `if let`
* `match`
* `unwrap()`

In [74]:
// extracting the content of Some(...)
if let Some(x) = tmp {
    println!("{}",x);
}
match tmp {
    Some(x) => println!("{}",x),
    None => println!("None"),
};
println!("Another way {}", tmp.unwrap())

839
839
Another way 839


()

Be careful with `unwrap()`, it will crash the program if the value is `None`.

In [75]:
// extracting the content of Some(...)
let tmp: Option<u32> = None;
if let Some(x) = tmp {
    println!("{}",x);
}
match tmp {
    Some(x) => println!("{}",x),
    None => println!("{:?}", tmp),
};
// Boom!!!!!
//println!("Another way {}", tmp.unwrap())

None


## Interesting related fact: Bertrand's postulate

There is always a prime number in $[k,2k]$. 
See [Prime Number Theorem](https://en.wikipedia.org/wiki/Prime_number_theorem)


# Enum `Option<T>`: useful methods

Check the variant
* `.is_some() -> bool`
* `.is_none() -> bool`

Get the value in `Some` or terminate with an error
* `.unwrap() -> T`
* `.expect(message) -> T`

Get the value in `Some` or a default value
* `.unwrap_or(default_value:T) -> T`

In [76]:
let x = Some(3);
x.is_some()

true

If exception, print a message.

In [77]:
// Try uncommenting line 3 or 4

//let x:Option<u32> = Some(3);
let x = None;
let y:u32 = x.expect("This should have been an integer!!!");
y

thread '<unnamed>' panicked at src/lib.rs:341:15:
This should have been an integer!!!
stack backtrace:
   0: _rust_begin_unwind
   1: core::panicking::panic_fmt
   2: core::option::expect_failed
   3: _run_user_code_61
   4: evcxr::runtime::Runtime::run_loop
   5: evcxr::runtime::runtime_hook
   6: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


A better way to handle this is to use `unwrap_or()`.

In [78]:
// Try uncommenting line 3 or 4

//let x = Some(3);
let x = None;
x.unwrap_or(0)

0

More details:
* https://doc.rust-lang.org/std/option/
* https://doc.rust-lang.org/std/option/enum.Option.html

## Enum `Result<T, E>`

Another built-in enum 
[`Result<T, E>`](https://doc.rust-lang.org/book/ch09-02-recoverable-errors-with-result.html?highlight=Result%3C#recoverable-errors-with-result)
in the standard library with two variants:

* `Ok(T)`
* `Err(E)`

Useful when you want to pass a solution or information about an error


In [79]:
fn divide(a:u32,b:u32) -> Result<u32,String> {
    match b {
        0 => Err(String::from("Division by zero")),
        _ => Ok(a / b)
    }
}

In [80]:
divide(3,0)

Err("Division by zero")

In [81]:
divide(2022,3)

Ok(674)

# Enum `Result<T, E>`: useful methods

Check the variant
* `.is_ok() -> bool`
* `.is_err() -> bool`

Get the value in `Ok` or terminate with an error
* `.unwrap() -> T`
* `.expect(message) -> T`

Get the value in `Ok` or a default value
* `.unwrap_or(default_value:T) -> T`

In [82]:
let r1 : Result<i32,()> = Ok(3);
// r1.is_err()
r1.is_ok()

true

In [83]:
r1.unwrap()

3

In [84]:
let r2 : Result<u32,()> = Err(());
let r3 : Result<u32,()> = Ok(123);
println!("r2: {}\nr3: {}",
    r2.unwrap_or(0),
    r3.unwrap_or(0));


r2: 0
r3: 123


More details:
* https://doc.rust-lang.org/std/result/
* https://doc.rust-lang.org/std/result/enum.Result.html

## Next Lecture (Thursday)

1. Traits (§10.2)
2. Collections (§8)
3. Vectors (§8.1)
4. Tests (§11)


## In-class poll

https://piazza.com/class/m5qyw6267j12cj/post/148
